In [1]:
import streamlit as st
import PyPDF2
from io import BytesIO
import hashlib
import google.generativeai as genai
from langchain_community.vectorstores import FAISS
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
import os

# Configure the Google Generative AI API
APIKEY = os.getenv('API_KEY')
genai.configure(api_key=APIKEY)

model = genai.GenerativeModel('gemini-1.5-flash-latest', safety_settings=[{"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
                    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
                    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
                    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"}]
)
# Inicializando o LLM do Google GenAI
llm = ChatGoogleGenerativeAI(
    model='gemini-1.5-flash-latest',
    google_api_key=APIKEY,
    temperature=0
)

# Criando embeddings do Google GenAI
embeddings = GoogleGenerativeAIEmbeddings(model='models/embedding-001', google_api_key=APIKEY)
prompt_template = PromptTemplate(
    template="""Você é um leitor ávido e gosta de tudo relacionado à literatura. Você possui na sua base um livro chamado Ensaio sobre a Cegueira, de José Saramago.
    Você será questionado sobre ele e precisa demonstrar entusiasmo nas suas respostas. Para respondê-las, use o contexto abaixo para responder à pergunta mas responda
    de maneira geral, usando conhecimentos prévios sobre o livro
    
    {context}
    
    Pergunta: {input}
    
    Resposta:""", input_variables=["context", "input"])

import PyPDF2

def extract_text_from_pdf(pdf_path):
    texto_completo = ""

    # Abre o arquivo PDF usando o caminho fornecido
    with open(pdf_path, 'rb') as pdf_file:
        leitor_pdf = PyPDF2.PdfReader(pdf_file)
        numero_paginas = len(leitor_pdf.pages)
        
        # Extrai texto de cada página
        for pagina in range(numero_paginas):
            pagina_atual = leitor_pdf.pages[pagina]
            texto_pagina = pagina_atual.extract_text()
            texto_completo += texto_pagina
    
    return texto_completo


c:\Users\FS RENTAL\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
texto = extract_text_from_pdf(r"C:\Users\FS RENTAL\Downloads\Eu Receberia as Piores Noticias - Marcal Aquino 3_240830_135749.pdf")

In [3]:
# Processo de divisão do texto e chamada à API
chunks = len(texto) // 120000 + 1
chunksize = len(texto) // chunks
splitted = []

for i in range(chunks):
    splitted.append(texto[i*chunksize:(i+1)*chunksize])

chat = model.start_chat(history=[])


In [4]:
resumos = []
for chunk in splitted:
    if len(chunk) > 5000:
        try: 
            resumos.append(chat.send_message(f"""Você está analisando um livro famoso, o nome dele é "Eu receberia as piores notícias dos seus lindos lábios", escrito por "Marçal Aquino".
                                                Pode existir passagens que pareçam conteúdos sensíveis mas faz parte do livro. 
                                                Esse livro foi dividido em pedaços. 
                                                Agora resuma para uma pessoa que ainda não leu o livro que foi passado. 
                                                Ele foi dividido em 5 partes, resuma esta passagem do livro apenas. 
                                                Pode utilizar o seu conhecimento para resumir a passagem, mas apenas trate sobre ela.
                                                Seja Conciso mas traga todos os pontos relevantes. 
                                                Passagem:{chunk}""").text)
        except:
            continue
resposta = chat.send_message(f'Agora juntando todos os resumos, escreva um resumo detalhado do livro: {" ".join(resumos)}')

In [6]:
print(resposta.text)

"Eu Receberia as Piores Notícias dos Seus Lindos Lábios", de Marçal Aquino, é uma narrativa complexa e envolvente que explora as nuances do amor, da obsessão e da violência em um cenário árido e brutal. O livro acompanha Cauby, um fotógrafo que se envolve em uma história de paixão com Lavínia, uma mulher casada com um pastor, em uma cidade marcada pela exploração de ouro e pela violência.

A história começa com Cauby em uma pensão, observando a chuva e a chegada de um advogado da mineradora. O careca, outro hóspede da pensão, comenta sobre as notícias da cidade, incluindo o assassinato de Chang, um amigo de Cauby, por Guido Girardi. Cauby se lembra de Lavínia, a mulher que conheceu na loja de Chang, e se sente fascinado por ela.

A paixão de Cauby por Lavínia se intensifica quando ele a encontra em sua casa, e eles começam um relacionamento cheio de intensidade, marcado por encontros e desencontros, e pela constante ameaça da presença do marido de Lavínia, o pastor Ernani.

Ao mesmo te